In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


In [ ]:
base_url = "https://terms.naver.com/list.naver?cid=51004&categoryId=51004&so=st3.asc&viewType=&categoryType=&page="
disease_links = []

# 예시: 첫 3페이지만 테스트용
for page in range(1, 3):
    url = base_url + str(page)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    links = soup.select(".info_area > .title > a")
    for link in links:
        href = "https://terms.naver.com" + link['href']
        disease_links.append(href)

print(f"총 {len(disease_links)}개의 질병 링크를 수집했습니다.")


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
def extract_disease_info(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    title = soup.select_one("h2.title").text.strip()

    sections = soup.select(".section_wrap .section")
    data = {
        "질병명": title,
        "정의": "",
        "원인": "",
        "증상": "",
        "진단법": "",
        "치료방법": ""
    }

    for sec in sections:
        header = sec.select_one("h3")
        if not header:
            continue
        h3 = header.text.strip()
        content = sec.get_text(separator=" ").strip().replace(h3, "").strip()

        # 정의
        if "개요-정의" in h3:
            data["정의"] = content
        elif "개요" == h3 and not data["정의"]:
            data["정의"] = content

        # 원인
        elif "원인" in h3:
            data["원인"] += content + " "

        # 증상
        elif "증상" in h3:
            data["증상"] += content + " "

        # 진단
        elif "진단" in h3:
            data["진단법"] += content + " "

        # 치료
        elif "치료" in h3:
            data["치료방법"] += content + " "

    return data


In [ ]:
all_data = []
for i, link in enumerate(disease_links):
    try:
        info = extract_disease_info(link)
        all_data.append(info)
        print(f"[{i+1}/{len(disease_links)}] 완료: {info['질병명']}")
        time.sleep(0.5)
    except Exception as e:
        print(f"[오류] {link} - {e}")


In [ ]:
df = pd.DataFrame(all_data)
df.to_csv("disease_info.csv", index=False, encoding="utf-8-sig")
print("CSV 저장 완료: disease_info.csv")
